# HW 2

In [1]:
import os
import requests
import threadpoolctl
import warnings
import typing as tp
from tqdm import tqdm
from pathlib import Path

import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization


from lightfm import LightFM


from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import (
    Precision,
    Recall,
    MAP,
    MeanInvUserFreq,
    NDCG,
    Serendipity,
    calc_metrics,
    AvgRecPopularity,
)

from rectools.models import (
    PopularModel,
    RandomModel,
    PureSVDModel,
    ImplicitALSWrapperModel,
    LightFMWrapperModel,
)

from rectools.models import implicit_knn
from implicit import nearest_neighbours
from rectools.model_selection import TimeRangeSplitter, cross_validate

warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid")

# For implicit ALS
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

In [2]:
RANDOM_STATE = 42
NUM_THREADS = 12

## Датасет KION

In [3]:
if not (
    Path("./data_original/interactions.csv").is_file()
    & Path("./data_original/items.csv").is_file()
    & Path("./data_original/users.csv").is_file()
):
    url = "https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip"
    req = requests.get(url, stream=True)

    with open("kion_train.zip", "wb") as fd:
        total_size_in_bytes = int(req.headers.get("Content-Length", 0))
        progress_bar = tqdm(
            desc="Downloading the kion dataset...",
            total=total_size_in_bytes,
            unit="iB",
            unit_scale=True,
        )
        for chunk in req.iter_content(chunk_size=2**20):
            progress_bar.update(len(chunk))
            fd.write(chunk)

In [4]:
# !unzip -f kion_train.zip -x '__MACOSX/*'

## <span style="color:blue">**Домашнее задание. (5 баллов)**</span>
**Задача: собрать итоговую модель/пайплайн, который одновременно даст**

<span style="color:blue">3 балла: **Precision@10 больше 0.03 и MeanInvUserFreq@10 ("novelty") больше 6</span>.**

<span style="color:blue">4 балла: **Precision@10 больше 0.032 и MeanInvUserFreq@10 ("novelty") больше 6.5</span>.**

<span style="color:blue">5 баллов: **Precision@10 больше 0.033 и MeanInvUserFreq@10 ("novelty") больше 6.9</span>.**

**Доп баллы 2: при пробитии порогов на 5 баллов максимальный novelty при precision от 0.33**

Список разрешённых моделей: EASE, ItemKNN из implicit, Popular, PopularInCategory, Transitions, PureSDV, BPR-MF, LogisticMF, iALS (+ фичи), LightFM (+ фичи)

Что также можно использовать: Fallback, тюнинг гипер-параметров, поиграться с весами интеракций в датасете / на инференсе, личные придуманные эвристики.
Можно также взять собственную модель с эвристиками, математикой 

Что нельзя использовать: ML и RecSys модели вне разрашённого списка. Чистая математика разрешена

Вывести в ноутбуке топ-3 модели по целевым метрикам.

PS. Тестируем на последней неделе


In [5]:
# Ваша схема валидации

# Загружаем данные
interactions = pd.read_csv(
    "./data_original/interactions.csv",
    parse_dates=[
        "last_watch_dt",
    ],
)
users = pd.read_csv("./data_original/users.csv")
items = pd.read_csv("./data_original/items.csv")

In [6]:
# Переименовываем колонки
interactions = interactions.rename(
    columns={
        "total_dur": Columns.Weight,
        "last_watch_dt": Columns.Datetime,
    }
)

In [7]:
interactions

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0


## Add features 

### For interactions


### For users

In [8]:
users.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [9]:
users.fillna("Unknown", inplace=True)

In [10]:
FEATURES_COLUMNS = ["age", "income", "sex", "kids_flg"]

user_features_dfs = []
for feature in FEATURES_COLUMNS:
    feature_df = users.reindex(columns=[Columns.User, feature])
    feature_df.columns = ["id", "value"]
    feature_df["feature"] = feature
    user_features_dfs.append(feature_df)
user_features_df = pd.concat(user_features_dfs, ignore_index=True)
user_features_df.head()

,id,value,feature
0,973171,age_25_34,age
1,962099,age_18_24,age
2,1047345,age_45_54,age
3,721985,age_45_54,age
4,704055,age_35_44,age


### For items

In [11]:
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [12]:
# Предварительно обратобтаем genres
items["genre"] = items["genres"].str.lower().str.split(", ")
genre_feature = items[[Columns.Item, "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [13]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
content_feature.head()

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type


In [14]:
country_feature = items.reindex(columns=[Columns.Item, "countries"])
country_feature.columns = ["id", "value"]
country_feature["feature"] = "countries"
country_feature.head()

,id,value,feature
0,10711,Испания,countries
1,2508,США,countries
2,10716,Канада,countries
3,7868,Великобритания,countries
4,16268,СССР,countries


In [15]:
year_feature = items.reindex(columns=[Columns.Item, "release_year"])
year_feature.columns = ["id", "value"]
year_feature["feature"] = "release_year"
year_feature.head()

,id,value,feature
0,10711,2002.0,release_year
1,2508,2014.0,release_year
2,10716,2011.0,release_year
3,7868,2015.0,release_year
4,16268,1978.0,release_year


In [16]:
items_features_df = pd.concat(
    [
        content_feature,
        genre_feature,
        country_feature,
        year_feature,
    ],
    ignore_index=True,
)
items_features_df.head()

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type


In [17]:
# Создаем датасет
# Измените датасет при желании

binary_interactions = interactions.copy()

# Добавим веса для фильмов
binary_interactions[Columns.Weight] = np.where(
    binary_interactions["watched_pct"] > 10, 2, 1
)
binary_interactions[Columns.Weight] = np.where(
    binary_interactions["watched_pct"] > 50, 4, binary_interactions[Columns.Weight]
)
binary_interactions[Columns.Weight] = np.where(
    binary_interactions["watched_pct"] > 75, 8, binary_interactions[Columns.Weight]
)
binary_interactions[Columns.Weight] = np.where(
    binary_interactions["watched_pct"] > 90, 10, binary_interactions[Columns.Weight]
)

current_dataset = Dataset.construct(
    interactions_df=binary_interactions,
    user_features_df=user_features_df,
    cat_user_features=["sex", "age", "income", "kids_flg"],
    item_features_df=items_features_df,
    cat_item_features=["genre", "content_type", "countries", "release_year"],
)

In [18]:
knn_impl_cos_k9 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.CosineRecommender(K=9)
)
knn_impl_cos_k10 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.CosineRecommender(K=10)
)
knn_impl_cos_k11 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.CosineRecommender(K=11)
)

knn_impl_tfidf_k7 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.TFIDFRecommender(K=7)
)

In [19]:
test_models = {
    "knn_impl_cos_k9": knn_impl_cos_k9,
    "knn_impl_cos_k10": knn_impl_cos_k10,  # leader
    "knn_impl_cos_k11": knn_impl_cos_k11,
    "knn_impl_tfidf_k7": knn_impl_tfidf_k7,
}  # Добавьте сюда любое количество моделей, которое вы тестируете

k = 10

metrics = {
    "precision": Precision(
        k
    ),  # Это целевая метрика для задания, остальные можно убрать если они не интересны
    # "ndcg": NDCG(k),
    # "serendipity": Serendipity(k),
    # "pop_bias": AvgRecPopularity(k, normalize=True),
    "MAP": MAP(k),
    "novelty": MeanInvUserFreq(
        k
    ),  # Это целевая метрика для задания, остальные можно убрать если они не интересны
}

splitter = TimeRangeSplitter(  # time-based валидация
    "7D",  # скользящее окно по 7 дней
    n_splits=1,  # 1 фолд (делаем для ускорения расчётов)
    filter_cold_users=True,  # только hot юзеры в тесте
    filter_cold_items=True,  # только hot айтемы в тесте
    filter_already_seen=True,  # дропаем из теста просмотры трейна
)

In [20]:
res = cross_validate(
    current_dataset,
    splitter,
    metrics,
    test_models,
    k=10,
    filter_viewed=True,
)
metrics_res = pd.DataFrame(res["metrics"])

# Результаты можно сохранить в csv для логирования экспериментов
metrics_res.to_csv("metric_result_final.csv", index=False)

In [21]:
metrics_res

,model,i_split,precision,MAP,novelty
0,knn_impl_cos_k9,0,0.032760,0.072988,7.071953
1,knn_impl_cos_k10,0,0.033809,0.074494,6.956733
2,knn_impl_cos_k11,0,0.033913,0.074496,6.869621
3,knn_impl_tfidf_k7,0,0.032329,0.073495,6.959991


Модель implicit_KNN CosineRecommener:

```python
knn_impl_cos_k10 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.CosineRecommender(K=10)
)
```
дала необходимые результаты по метрикам.